# Move the Data to Pandas

In [1]:
import sys
sys.path.insert(0, '../modules')
%pylab inline
from pprint import pprint
from tracking import HoughSpace
from scipy import sparse
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.ticker as mtick
from hits import CyDetHits, CTHHits, CDCHits, FlatHits
import data_tools as dts
import visualizations as viz
from root_numpy import list_branches, list_trees
from collections import OrderedDict
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


/home/five4three2/development/miniconda3/envs/tfy3_again/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.14.3 is currently installed but you installed root_numpy against numpy 1.12.1. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [2]:
a_file = "/home/five4three2/development/ICEDUST/track-finding-yandex"+\
         "/data/MC4q/8e6_pot/bunch_train_6/root/MC4q_bkg_f5_b6_t-100_2000.root"

In [3]:
all_branches = OrderedDict()
# Get the two hit trees
trees = [tree for tree in list_trees(a_file) if "Hit" in tree]
# Get the branches for each tree
for tree in trees:
    branches = [branch for branch in list_branches(a_file, treename=tree)]
    for filter_br in ["CDCHit.fDetectedTime",
                      "CDCHit.fCharge",
                      "CDCHit.fEventNumber",
                      "CDCHit.fIsSig",
                      'CTHHit.fMCPos.fE',
                      'CTHHit.fCharge',
                      'CTHHit.fEventNumber',
                      'CTHHit.fIsSig']:
        branches = [branch for branch in branches if filter_br not in branch]
    all_branches[tree[:3]] = branches
print(trees)

['CDCHitTree', 'CTHHitTree']


TClass::Init:0: RuntimeWarning: no dictionary for class COMET::ICOMETEvent is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::IDataVector is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::IData is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::IDatum is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::ICOMETContext is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::ISHAHashValue is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::IAlignmentId is available
TClass::Init:0: RuntimeWarning: no dictionary for class COMET::ICOMETEvent::Header is available


In [4]:
#flat_hits.data.index.levels[flat_hits.data.index.names.index(flat_hits.event_index_name
these_cuts = ["500","Trig","Track"]
these_cuts = dts.data_get_cuts(these_cuts, signal=False)                           

Using cuts
{ 'CDC': 'CDCHit.fDetectedTime < 1620 && CDCHit.fDetectedTime > 500 && 1 == 1 '
         '&& 1 == 1',
  'CTH': 'CTHHit.fMCPos.fE < 1170 && CTHHit.fMCPos.fE > 500 && 1 == 1 && 1 == '
         '1'}


In [10]:
cth_hits = CTHHits(a_file, trees[1], selection=these_cuts["CTH"], branches=all_branches["CTH"])
cdc_hits = CDCHits(a_file, trees[0], selection=these_cuts["CDC"], branches=all_branches["CDC"])
MB = 1024*1024
print("Size %d MB" % (sys.getsizeof(cdc_hits.data)/MB) )
print("Size %d MB" % (sys.getsizeof(cth_hits.data)/MB) )
print(np.sum(cdc_hits.data.memory_usage(deep=True).values)/MB)
print(np.sum(cth_hits.data.memory_usage(deep=True).values)/MB)
cdc_columns = ["".join(col.split(".f")[1:]) for col in cdc_hits.data.columns.values]
cth_columns = ["".join(col.split(".f")[1:]) for col in cth_hits.data.columns.values]
pprint([col for col in cdc_columns if col not in cth_columns])
pprint([col for col in cth_columns if col not in cdc_columns])
cdc_hits.data.columns = cdc_columns
cth_hits.data.columns = cth_columns

Size 1228 MB
Size 8 MB
1228.4559526443481
8.707139015197754
['DetectedTime',
 'DriftDist',
 'DriftTime',
 'MCMomX',
 'MCMomY',
 'MCMomZ',
 'TurnID']
['Counter', 'IsLG', 'IsSc', 'Module', 'NContrib', 'vol_id']


In [13]:
cydet_hits = pd.concat([cdc_hits.data, cth_hits.data], keys=["CDC","CTH"]).to_sparse()
print(np.sum(cydet_hits.memory_usage(deep=True).values)/MB)

/home/five4three2/development/miniconda3/envs/tfy3_again/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


1283.0128145217896
